In [ ]:
import pandas as pd

# Ler a lista de colunas específicas do arquivo 'colunas_especificas.txt'
with open("colunas_especificas.txt", 'r') as fin:
    cols = fin.readlines()
    cols_especificas = [c.strip() for c in cols]

# Defina as colunas para ordenação
id_cols = ['ISIN', 'Citi Code', 'EPT Reference Language']

# Combine as colunas necessárias para leitura (evita ler colunas desnecessárias)
cols_to_read = cols_especificas + [col for col in id_cols if col not in cols_especificas]

# Definir o tamanho do chunk (número de linhas a serem lidas)
# Se chunk_size for None, o script lerá o arquivo inteiro
chunk_size = 1000  # Defina para um número inteiro, por exemplo, 1000, para ler apenas 1000 linhas

for xx in range(1, 31):
    xx_formatted = f"{xx:02d}"
    input_filename = f"Quali_202406{xx_formatted}.csv"
    output_filename = f"prio_{xx_formatted}_sorted.csv"

    try:
        if chunk_size is not None:
            # Ler apenas as primeiras 'chunk_size' linhas
            df = pd.read_csv(input_filename, usecols=cols_to_read, nrows=chunk_size)
        else:
            # Ler o arquivo inteiro
            df = pd.read_csv(input_filename, usecols=cols_to_read)

        # Verificar se as colunas de ordenação estão presentes
        missing_cols = [col for col in id_cols if col not in df.columns]
        if missing_cols:
            print(f"As seguintes colunas para ordenação estão ausentes no arquivo {input_filename}: {missing_cols}")
            continue

        # Ordenar o DataFrame pelas colunas especificadas
        df_sorted = df.sort_values(by=id_cols)

        # Salvar o DataFrame filtrado e ordenado em um novo arquivo CSV
        df_sorted.to_csv(output_filename, index=False)

        print(f"Arquivo '{output_filename}' salvo com sucesso.")

    except FileNotFoundError:
        print(f"O arquivo '{input_filename}' não foi encontrado. Pulando para o próximo arquivo.")
        continue
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo '{input_filename}': {e}")
        continue


In [ ]:
import pandas as pd
import sys

# Defina as colunas identificadoras
id_cols = ['ISIN', 'Citi Code', 'EPT Reference Language']

# Lista para armazenar os DataFrames com a coluna 'data' adicionada
dfs_with_date = []

total_chunks = 30  # Total de chunks a serem processados

for xx in range(1, total_chunks + 1):
    xx_formatted = f"{xx:02d}"
    filename = f"prio_{xx_formatted}_sorted.csv"
    
    try:
        # Ler o arquivo 'prio_xx_sorted.csv'
        df = pd.read_csv(filename)
        
        # Adiciona a coluna 'data' com o valor do dia no formato 'xx/06/2024'
        df['data'] = f"{xx_formatted}/06/2024"
        
        # Certifique-se de que as colunas identificadoras estão presentes
        missing_cols = [col for col in id_cols if col not in df.columns]
        if missing_cols:
            print(f"\nAs seguintes colunas estão ausentes no arquivo {filename}: {missing_cols}")
            continue
        
        dfs_with_date.append(df)
        
        # Imprimir o progresso e apagar a linha anterior
        sys.stdout.write(f"\rProcessando chunk {xx} de {total_chunks}")
        sys.stdout.flush()
        
    except FileNotFoundError:
        print(f"\nO arquivo '{filename}' não foi encontrado. Pulando para o próximo arquivo.")
        continue
    except Exception as e:
        print(f"\nOcorreu um erro ao processar o arquivo '{filename}': {e}")
        continue

print("\nTodos os chunks foram processados.")

# Combina todos os DataFrames em um único DataFrame
combined_df = pd.concat(dfs_with_date, ignore_index=True)

# Converter a coluna 'data' para datetime para facilitar a ordenação
combined_df['data'] = pd.to_datetime(combined_df['data'], format='%d/%m/%Y')

# Ordena o DataFrame pelos identificadores e pela data
combined_df.sort_values(by=id_cols + ['data'], inplace=True)

# Lista para armazenar as mudanças detectadas
changes = []

# Definir as colunas a serem comparadas (todas exceto as identificadoras e 'data')
value_cols = [col for col in combined_df.columns if col not in id_cols + ['data']]

# Agrupa o DataFrame pelos identificadores
grouped = combined_df.groupby(id_cols)

for name, group in grouped:
    group = group.sort_values(by='data').reset_index(drop=True)
    for i in range(1, len(group)):
        current_row = group.loc[i]
        previous_row = group.loc[i - 1]
        data_current = current_row['data']
        data_previous = previous_row['data']
        
        # Ignorar mudanças onde a data atual é o primeiro dia
        if data_current == pd.to_datetime('01/06/2024', format='%d/%m/%Y'):
            continue
        
        for col in value_cols:
            value_current = current_row[col]
            value_previous = previous_row[col]
            # Verifica se os valores são diferentes, considerando valores nulos
            if pd.isnull(value_current) and pd.isnull(value_previous):
                continue  # Ambos são nulos, não há mudança
            elif pd.isnull(value_current) != pd.isnull(value_previous):
                # Um valor é nulo e o outro não
                changes.append({
                    'campo': col,
                    'valor_antigo': value_previous,
                    'valor_novo': value_current,
                    'data': data_current.strftime('%d/%m/%Y')
                })
            elif value_current != value_previous:
                changes.append({
                    'campo': col,
                    'valor_antigo': value_previous,
                    'valor_novo': value_current,
                    'data': data_current.strftime('%d/%m/%Y')
                })

print("\nDetecção de mudanças concluída.")

# Cria um DataFrame com as mudanças
changes_df = pd.DataFrame(changes)

# Converter a coluna 'data' para datetime para ordenação
changes_df['data'] = pd.to_datetime(changes_df['data'], format='%d/%m/%Y')

# Ordena o DataFrame 'changes_df' pela coluna 'data'
changes_df.sort_values(by='data', inplace=True)

# Opcional: Converte a coluna 'data' de volta para string no formato desejado
changes_df['data'] = changes_df['data'].dt.strftime('%d/%m/%Y')

# Seleciona apenas as colunas desejadas
changes_df = changes_df[['campo', 'valor_antigo', 'valor_novo', 'data']]

# Exibe o DataFrame com as mudanças ordenadas por data
print("\nPrimeiras linhas do DataFrame de mudanças:")
print(changes_df.head())
